In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from development.datasets.osdg_dataset import load_osdg_data
from development.datasets.uclmodules_dataset import load_uclmodules_data
from development.datasets.videscription_dataset import load_videscription_data
from development.train_model import fine_tune_transformer
from development.models.Bert import Bert
from development.models.RoBERTa import RoBERTa
from development.models.ALBERT import ALBERT

In [3]:
with open('config.json', 'r') as file:
    CONFIG = json.load(file)
    dev_config = CONFIG['development']


In [4]:
osdg_data = load_osdg_data(dev_config['osdg_data_path'], training=True)

In [ ]:
ucl_data = load_uclmodules_data(dev_config['uclmodules_data_path'])

In [ ]:
videscription_data = load_videscription_data(
    dev_config['videscription_data_path']
)

In [5]:
bert = Bert()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
result = fine_tune_transformer(
    bert.model,
    bert.tokenizer,
    bert.tokenizer_args,
    data=osdg_data,
    dev_config=dev_config
)

In [ ]:
cls_report = bert.evaluate(bert.model, bert.tokenizer, osdg_data['test'])
print(cls_report)

In [ ]:
cls_report = bert.evaluate(bert.model, bert.tokenizer, ucl_data, add_one=True)
print(cls_report)

In [ ]:
roberta = RoBERTa()

In [ ]:
result_2 = fine_tune_transformer(
    roberta.model,
    roberta.tokenizer,
    roberta.tokenizer_args,
    data=osdg_data,
    dev_config=dev_config
)

In [ ]:
cls_report = roberta.evaluate(
    roberta.model,
    roberta.tokenizer,
    osdg_data['test']
)
print(cls_report)

In [ ]:
cls_report = roberta.evaluate(
    roberta.model,
    roberta.tokenizer,
    ucl_data,
    add_one=True
)
print(cls_report)

In [17]:
albert = ALBERT()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model 

In [ ]:
result_3 = fine_tune_transformer(
    albert.model,
    albert.tokenizer,
    albert.tokenizer_args,
    data=osdg_data,
    dev_config=dev_config
)

In [ ]:
cls_report = albert.evaluate(albert.model, albert.tokenizer, osdg_data['test'])
print(cls_report)

In [ ]:
cls_report = albert.evaluate(
    albert.model,
    albert.tokenizer,
    ucl_data,
    add_one=True
)
print(cls_report)